<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Sebastian Raschka의 도서 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>를 위한 보조 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 3 어텐션 메커니즘 코딩하기

이 노트북에서 사용하는 패키지:

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.8.0


- 이 장은 LLM의 엔진인 어텐션 메커니즘을 다룹니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 긴 시퀀스 모델링의 문제점

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- 트랜스포머 이전에는 기계번역에 인코더–디코더 RNN이 널리 쓰였습니다
- 인코더는 원문 토큰 시퀀스를 처리하며, 은닉 상태(중간 신경망 계층)를 사용해 전체 입력 시퀀스의 압축 표현을 만듭니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 어텐션 메커니즘으로 데이터 의존성 포착하기

- 어텐션 메커니즘을 통해 디코더는 모든 입력 토큰에 선택적으로 접근할 수 있으며, 이는 특정 출력 토큰 생성에 일부 입력 토큰이 더 중요함을 의미합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- 트랜스포머의 셀프 어텐션은 시퀀스의 각 위치가 동일 시퀀스 내 다른 모든 위치와 상호작용하며 중요도를 판단해 입력 표현을 향상시키는 기법입니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 셀프 어텐션으로 입력의 다른 부분에 주의 기울이기

### 3.3.1 훈련 가능한 가중치 없는 간단한 셀프 어텐션 메커니즘

- 이 절에서는 훈련 가능한 가중치가 없는 단순화된 셀프 어텐션 변형을 설명합니다
- 이는 개념 설명을 위한 것으로, 트랜스포머에서 실제로 사용하는 어텐션은 아닙니다
- 다음 3.3.2 절에서 이 단순 버전을 확장하여 실제 셀프 어텐션을 구현합니다
- 입력 시퀀스 $x^{(1)}$부터 $x^{(T)}$가 주어졌다고 합시다
  - 입력은 이미 토큰 임베딩으로 변환된 텍스트(예: "Your journey starts with one step")입니다
  - 예를 들어 $x^{(1)}$은 단어 "Your"를 나타내는 \(d\)-차원 벡터입니다
- **목표:** 시퀀스의 각 요소 $x^{(i)}$에 대해 컨텍스트 벡터 $z^{(i)}$를 계산합니다(\(z\)와 \(x\)의 차원은 동일)
    - 컨텍스트 벡터 $z^{(i)}$는 $x^{(1)}$부터 $x^{(T)}$까지 입력의 가중합입니다
    - 컨텍스트 벡터는 특정 입력에 대해 맥락적으로 정의됩니다
      - 임의의 입력 토큰을 나타내는 $x^{(i)}$ 대신, 두 번째 입력 $x^{(2)}$를 보겠습니다
      - 마찬가지로 $z^{(i)}$ 대신 두 번째 컨텍스트 벡터 $z^{(2)}$를 보겠습니다
      - $z^{(2)}$는 $x^{(2)}$에 대한 가중치로 $x^{(1)}$부터 $x^{(T)}$까지 모든 입력의 가중합입니다
      - 어텐션 가중치는 $z^{(2)}$ 계산 시 각 입력 요소가 얼마나 기여하는지를 정하는 가중치입니다
      - 요약하면, $z^{(2)}$는 $x^{(2)}$를 기반으로 하되 관련 입력 정보가 반영된 수정본으로 볼 수 있습니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- (이 그림의 숫자는 시각적 혼잡을 줄이기 위해 소수점 첫째 자리까지만 표시했습니다. 다른 그림에서도 동일하게 값이 생략될 수 있습니다)

- 관례적으로, 정규화되지 않은 어텐션 가중치는 **어텐션 스코어(attention scores)**, 합이 1이 되도록 정규화된 값은 **어텐션 가중치(attention weights)** 라 부릅니다

- 아래 코드는 위 도해를 단계별로 따라갑니다
- **1단계:** 정규화되지 않은 어텐션 스코어 $\omega$ 계산
- 두 번째 입력 토큰을 쿼리로 사용한다고 하면, 즉 $q^{(2)} = x^{(2)}$일 때, 도트 곱으로 어텐션 스코어를 계산합니다:
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- 여기서 $\omega$는 정규화되지 않은 어텐션 스코어를 나타내는 그리스 문자입니다
    - $\omega_{21}$의 아래첨자 "21"은 입력 2를 쿼리로 하여 입력 1에 대해 계산했음을 의미합니다

- 다음은 3차원 임베딩으로 변환된 입력 문장 예시입니다(지면상 작은 임베딩 차원을 사용합니다):

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # 'Your' 토큰 (x^1)
   [0.55, 0.87, 0.66], # 'journey' 토큰 (x^2)
   [0.57, 0.85, 0.64], # 'starts' 토큰 (x^3)
   [0.22, 0.58, 0.33], # 'with' 토큰 (x^4)
   [0.77, 0.25, 0.10], # 'one' 토큰 (x^5)
   [0.05, 0.80, 0.55]] # 'step' 토큰 (x^6)
)

- 이 책에서는 관례적으로 학습 예시를 행, 특성값을 열로 표현합니다. 위 텐서에서 각 행은 단어, 각 열은 임베딩 차원을 의미합니다

- 이 절의 핵심 목표는 두 번째 입력 $x^{(2)}$를 쿼리로 사용해 컨텍스트 벡터 $z^{(2)}$를 계산하는 과정을 보여주는 것입니다

- 아래 그림은 첫 단계로, $x^{(2)}$와 다른 모든 입력 간 도트 곱으로 어텐션 스코어 $\omega$를 계산하는 과정을 나타냅니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- 입력 시퀀스의 두 번째 요소 $x^{(2)}$를 예시로 컨텍스트 벡터 $z^{(2)}$를 계산합니다. 이후 모든 컨텍스트 벡터로 일반화합니다
- 첫 단계는 쿼리 $x^{(2)}$와 다른 모든 입력 토큰 간 도트 곱으로 정규화되지 않은 어텐션 스코어를 계산하는 것입니다:

In [3]:
query = inputs[1]  # 두 번째 입력 토큰을 쿼리로 사용

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # 도트 곱(1차원 벡터이므로 전치가 필요 없음)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 참고: 도트 곱은 두 벡터를 요소별로 곱한 뒤 그 합을 구하는 연산의 간단한 표기입니다:

In [4]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **2단계:** 정규화되지 않은 어텐션 스코어(\"오메가\", $\omega$)를 합이 1이 되도록 정규화합니다
- 아래는 해석과 훈련 안정성 측면에서 유용한, 간단한 정규화 예시입니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [5]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 그러나 실무에서는 극단값에 강하고 기울기 특성이 좋은 소프트맥스를 사용하는 것이 일반적이며 권장됩니다
- 아래는 요소들의 합이 1이 되도록 정규화하는 소프트맥스의 단순 구현 예시입니다:

In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 위의 순진한 구현은 입력 값이 매우 크거나 작을 때 오버플로/언더플로 때문에 수치적으로 불안정해질 수 있습니다
- 따라서 실제로는 성능이 최적화된 PyTorch의 softmax 구현을 사용하는 것이 좋습니다:

In [7]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- **3단계:** 임베딩된 입력 $x^{(i)}$에 어텐션 가중치를 곱해 합하여 컨텍스트 벡터 $z^{(2)}$를 계산합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [8]:
query = inputs[1] # 두 번째 입력 토큰을 쿼리로 사용

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 모든 입력 토큰에 대한 어텐션 가중치 계산

#### 모든 입력 시퀀스 토큰으로 일반화하기:

- 위에서는 입력 2에 대한 어텐션 가중치와 컨텍스트 벡터를 계산했습니다(아래 그림의 강조된 행 참조)
- 이제 이 계산을 전체 어텐션 가중치와 컨텍스트 벡터로 일반화합니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- (이 그림의 숫자는 시각적 혼잡을 줄이기 위해 소수점 둘째 자리까지만 표시했으며, 각 행의 값은 1.0 또는 100%가 되도록 합산됩니다. 다른 그림들도 마찬가지로 자릿수가 생략될 수 있습니다)

- 셀프 어텐션은 먼저 어텐션 스코어를 계산하고, 이를 정규화해 합이 1인 어텐션 가중치를 얻습니다
- 이후 이 가중치를 사용해 입력의 가중합으로 컨텍스트 벡터를 계산합니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- 이전 **1단계**를 모든 쌍에 적용해 정규화되지 않은 어텐션 스코어 행렬을 계산합니다:

In [9]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 위 연산은 행렬 곱셈으로 더 효율적으로 동일하게 계산할 수 있습니다:

In [10]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 이전 **2단계**와 마찬가지로, 각 행의 합이 1이 되도록 정규화합니다:

In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 각 행의 값이 실제로 1이 되는지 빠르게 확인합니다:

In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 이전 **3단계**를 적용해 모든 컨텍스트 벡터를 계산합니다:

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 간단한 확인: 위 결과의 두 번째 행에서 이전에 계산한 컨텍스트 벡터 $z^{(2)} = [0.4419, 0.6515, 0.5683]$를 확인할 수 있습니다:

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 훈련 가능한 가중치를 사용한 셀프 어텐션 구현

- 이 절에서 개발하는 셀프 어텐션이 본 장과 책의 큰 흐름 속에서 어디에 위치하는지 개념적으로 보여줍니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 어텐션 가중치를 단계별로 계산하기

- 이 절에서는 원조 트랜스포머, GPT 계열, 대부분의 LLM에서 사용하는 셀프 어텐션 메커니즘을 구현합니다
- 이 메커니즘은 스케일드 도트 곱 어텐션(scaled dot-product attention)이라고도 합니다
- 핵심 아이디어는 이전과 유사합니다:
  - 특정 입력 요소에 대해 입력 벡터들의 가중합으로 컨텍스트 벡터를 계산합니다
  - 이를 위해 어텐션 가중치가 필요합니다
- 차이점은 크지 않지만 중요합니다:
  - 학습 중 업데이트되는 가중치 행렬의 도입이 핵심 차이입니다
  - 이 학습 가능한 가중치 행렬 덕분에 모델(특히 어텐션 모듈)이 "좋은" 컨텍스트 벡터를 학습할 수 있습니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- 셀프 어텐션을 단계별로 구현하면서, 먼저 세 가지 가중치 행렬 $W_q$, $W_k$, $W_v$를 도입합니다
- 이 세 행렬은 임베딩된 입력 $x^{(i)}$를 각각 쿼리, 키, 값 벡터로 선형 사상합니다:

  - 쿼리 벡터: $q^{(i)} = x^{(i)}\,W_q $
  - 키 벡터: $k^{(i)} = x^{(i)}\,W_k $
  - 값 벡터: $v^{(i)} = x^{(i)}\,W_v $


- 입력 $x$와 쿼리 $q$의 임베딩 차원은 설계에 따라 같을 수도, 다를 수도 있습니다
- GPT 계열에서는 보통 입력과 출력 차원이 같지만, 여기서는 계산을 쉽게 따라가기 위해 입력과 출력 차원을 다르게 설정합니다:

In [15]:
x_2 = inputs[1] # 두 번째 입력 요소
d_in = inputs.shape[1] # 입력 임베딩 크기(d=3)
d_out = 2 # 출력 임베딩 크기(d=2)

- 아래에서는 세 가지 가중치 행렬을 초기화합니다. 설명을 간단히 하기 위해 출력이 지저분해지는 것을 막고자 `requires_grad=False`로 설정했지만, 실제 학습에서는 이 값을 `True`로 두어 가중치를 업데이트해야 합니다

In [16]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 다음으로 쿼리, 키, 밸류 벡터를 계산합니다:

In [17]:
query_2 = x_2 @ W_query # _2는 두 번째 입력 요소를 기준으로 했다는 뜻
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- 아래에서 보듯 6개의 입력 토큰을 3차원 임베딩 공간에서 2차원 임베딩 공간으로 성공적으로 사영했습니다:

In [18]:
keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 다음 단계(**단계 2**)에서는 쿼리와 각 키 벡터의 도트 곱을 통해 정규화되지 않은 어텐션 스코어를 계산합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [19]:
keys_2 = keys[1] # 파이썬은 인덱스를 0부터 시작
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- 입력이 6개이므로 해당 쿼리 벡터에 대한 어텐션 스코어도 6개가 나옵니다:

In [20]:
attn_scores_2 = query_2 @ keys.T # 주어진 쿼리에 대한 모든 어텐션 스코어
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- 다음 **단계 3**에서는 앞서 사용한 softmax 함수를 이용해 (합이 1이 되도록 정규화된) 어텐션 가중치를 계산합니다
- 이전과의 차이점은 이제 어텐션 스코어를 임베딩 차원의 제곱근, 즉 $\sqrt{d_k}$(`d_k**0.5`)로 나누어 스케일링한다는 점입니다:

In [21]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

- **단계 4**에서는 이제 두 번째 쿼리 벡터에 대한 컨텍스트 벡터를 계산합니다:

In [22]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 간결한 SelfAttention 클래스 구현

- 지금까지의 내용을 합쳐 셀프 어텐션 메커니즘을 다음과 같이 구현할 수 있습니다:

In [23]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # ω (어텐션 스코어)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- PyTorch의 `Linear` 계층을 사용하면 위 구현을 더 간결하게 만들 수 있으며, 바이어스를 끄면 행렬 곱과 동일한 연산을 수행합니다
- 또한 `nn.Linear`를 사용하면 권장 초기화 방식이 자동으로 적용되어 우리가 직접 `nn.Parameter(torch.rand(...`로 정의했을 때보다 학습이 더 안정적입니다

In [24]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- `SelfAttention_v1`과 `SelfAttention_v2`는 서로 다른 초기 가중치를 사용하므로 출력이 다르게 나타난다는 점을 유의하세요

## 3.5 인과적 어텐션으로 미래 단어 숨기기

- 인과 어텐션에서는 대각선 위의 어텐션 가중치를 마스킹하여, 주어진 입력에 대해 컨텍스트 벡터를 계산할 때 미래 토큰을 활용하지 못하도록 합니다

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 인과적 어텐션 마스크 적용하기

- 이번 절에서는 앞에서 만든 셀프 어텐션을 인과 셀프 어텐션으로 확장합니다
- 인과 셀프 어텐션은 시퀀스의 특정 위치를 예측할 때 해당 위치 이전의 출력에만 의존하고, 이후 위치에는 의존하지 않도록 보장합니다
- 쉽게 말해, 다음 단어를 예측할 때는 지금까지 등장한 단어들만 참고하도록 강제하는 것입니다
- 이를 위해 각 토큰에 대해 현재 위치 이후에 등장하는 미래 토큰들을 마스킹합니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- 인과 셀프 어텐션을 설명하고 구현하기 위해 앞 절에서 계산한 어텐션 스코어와 가중치를 활용해 보겠습니다:

In [25]:
# 편의를 위해 이전 절에서 사용한 쿼리·키 가중치 행렬을 다시 활용합니다
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 미래의 어텐션 가중치를 가리는 가장 간단한 방법은 PyTorch의 `tril` 함수로 주대각선 이하를 1, 주대각선 위를 0으로 설정한 마스크를 만드는 것입니다:

In [26]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 그런 다음 이 마스크를 어텐션 가중치에 곱해 대각선 위의 값들을 0으로 만들 수 있습니다:

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 그러나 위처럼 소프트맥스 이후에 마스크를 적용하면, 소프트맥스가 만든 확률 분포가 깨지게 됩니다
- 소프트맥스는 출력 값의 합이 항상 1이 되도록 만듭니다
- 소프트맥스 이후에 마스킹하면 다시 합이 1이 되도록 정규화해야 하므로 과정이 복잡해지고, 의도치 않은 결과가 발생할 수 있습니다

- 각 행의 합이 1이 되도록 다음과 같이 어텐션 가중치를 다시 정규화할 수 있습니다:

In [28]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 사실 이제 인과 어텐션 코드는 완성됐지만, 같은 효과를 더 효율적으로 내는 방법을 간단히 살펴보겠습니다
- 대각선 위의 어텐션 가중치를 0으로 만든 뒤 재정규화하는 대신, 소프트맥스에 들어가기 전 정규화되지 않은 어텐션 스코어를 음의 무한대로 마스킹할 수 있습니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [29]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 아래 결과를 보면 각 행의 어텐션 가중치 합이 다시 1이 된 것을 확인할 수 있습니다:

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 드롭아웃으로 추가 어텐션 가중치 마스킹하기

- 또한 학습 중 과적합을 줄이기 위해 드롭아웃도 적용합니다
- 드롭아웃은 여러 위치에 적용할 수 있습니다:
  - 예를 들어 어텐션 가중치를 계산한 직후에 적용할 수 있고,
  - 어텐션 가중치와 밸류 벡터를 곱한 이후에 적용할 수도 있습니다
- 여기서는 어텐션 가중치를 구한 뒤 드롭아웃 마스크를 적용하는데, 실무에서 가장 흔한 방식이기 때문입니다

- 이번 예제에서는 드롭아웃 비율을 50%로 설정해 어텐션 가중치의 절반을 무작위로 마스킹합니다. (나중에 GPT 모델을 학습할 때는 0.1이나 0.2처럼 더 작은 비율을 사용할 예정입니다)

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- 드롭아웃 비율을 0.5(50%)로 사용하면 제거되지 않은 값들은 1/0.5 = 2배로 스케일됩니다
- 스케일링은 1 / (1 - `dropout_rate`) 공식을 따릅니다

In [31]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # 드롭아웃 비율 50%
example = torch.ones(6, 6) # 모든 값이 1인 행렬 생성

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [32]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- 드롭아웃 결과는 운영체제에 따라 다르게 보일 수 있습니다. 이러한 차이에 대한 자세한 설명은 [PyTorch 이슈 트래커](https://github.com/pytorch/pytorch/issues/121595)에서 확인할 수 있습니다.

### 3.5.3 컴팩트한 인과적 어텐션 클래스 구현하기

- 이제 인과 마스크와 드롭아웃 마스크를 포함한 실제 셀프 어텐션 구현을 만들어 보겠습니다
- 또한 2장에서 작성한 데이터 로더가 만들어 내는 배치 출력을 처리할 수 있도록 둘 이상의 입력을 포함한 배치도 대응하게 구현해야 합니다
- 간단한 예시로 이러한 배치를 흉내 내기 위해 입력 문장 예제를 복제하겠습니다:

In [33]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 두 개의 입력 각각 6개 토큰, 토큰 임베딩 차원은 3

torch.Size([2, 6, 3])


In [34]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # 추가된 부분
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # 추가된 부분

    def forward(self, x):
        b, num_tokens, d_in = x.shape # 새로운 배치 차원 b
        # `num_tokens`가 `context_length`를 넘으면 아래 마스크 생성에서 오류가 발생
        # 마스크를 만드는 부분에서 문제가 생김
        # 실제로는 4~7장에서 구현할 LLM이 입력이 `context_length`를 넘지 않도록 보장하므로 걱정하지 않아도 됨
        
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # 전치 대상 변경
        attn_scores.masked_fill_(  # 추가된 부분(_ 접미사는 제자리 연산)
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # 배치 토큰 수가 지원하는 컨텍스트 길이보다 짧은 경우를 고려해 `:num_tokens`로 슬라이스
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # 추가된 부분

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 드롭아웃은 학습 단계에만 적용되고 추론 단계에서는 적용되지 않는다는 점을 기억하세요

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 단일 헤드 어텐션을 멀티헤드 어텐션으로 확장하기

### 3.6.1 여러 단일 헤드 어텐션 레이어 쌓기

- 아래는 앞서 구현한 셀프 어텐션을 요약한 그림입니다(간결하게 표현하기 위해 인과/드롭아웃 마스크는 생략했습니다)

- 이는 싱글 헤드 어텐션이라고도 부릅니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- 여러 개의 싱글 헤드 어텐션 모듈을 쌓아 올리면 멀티헤드 어텐션 모듈을 만들 수 있습니다:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- 멀티헤드 어텐션의 핵심 아이디어는 서로 다른 학습된 선형 변환을 사용해 어텐션 연산을 병렬로 여러 번 수행하는 것입니다. 이렇게 하면 모델이 서로 다른 표현 부분 공간의 정보를 위치별로 동시에 살펴볼 수 있습니다.

In [35]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # 토큰 개수와 동일
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 위 구현에서 임베딩 차원이 4인 이유는 키·쿼리·밸류 벡터와 컨텍스트 벡터의 임베딩 차원을 모두 `d_out=2`로 설정했기 때문입니다. 어텐션 헤드가 2개이므로 출력 임베딩 차원은 2*2=4가 됩니다.

### 3.6.2 가중치 분할을 통한 멀티헤드 어텐션 구현

- 위 구현은 앞서 만든 `CausalAttention`을 여러 번 감싸는 직관적이고 완전한 멀티헤드 어텐션 예시이지만, 동일한 기능을 하는 독립형 `MultiHeadAttention` 클래스를 작성할 수도 있습니다.

- 이 독립형 `MultiHeadAttention` 클래스에서는 개별 어텐션 헤드를 단순히 이어 붙이지 않습니다.
- 대신 하나의 `W_query`, `W_key`, `W_value` 가중치 행렬을 만들고 이를 각 어텐션 헤드에 맞게 분할합니다:

In [36]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # 원하는 출력 차원과 맞추기 위해 투영 차원을 축소

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # 헤드 출력을 결합하는 선형 계층
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # `CausalAttention`과 마찬가지로 `num_tokens`가 `context_length`를 초과하면 
        # 아래에서 마스크를 만들 때 오류가 발생한다는 뜻임. 
        # 하지만 4~7장에서 구현할 LLM이 입력이 `context_length`를 초과하지 않도록 처리하므로 실제로는 문제가 되지 않음. 
        

        keys = self.W_key(x) # 형태: (배치, 토큰 수, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # `num_heads` 차원을 추가해 행렬을 암묵적으로 분리
        # 마지막 차원을 펼쳐 (b, num_tokens, d_out)을 (b, num_tokens, num_heads, head_dim)으로 변환
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 전치: (b, num_tokens, num_heads, head_dim) → (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # 인과 마스크를 적용해 스케일된 점곱 어텐션(셀프 어텐션)을 계산
        attn_scores = queries @ keys.transpose(2, 3)  # 헤드마다 도트 곱 수행

        # 원래 마스크를 토큰 수에 맞게 잘라 불리언으로 변환
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # 마스크를 이용해 어텐션 스코어를 채움
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # 형태: (배치, 토큰 수, 헤드 수, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # self.d_out = self.num_heads * self.head_dim이 되도록 헤드를 결합
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # 선택적 투영

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 위 클래스는 `MultiHeadAttentionWrapper`를 보다 효율적으로 다시 작성한 버전이라는 점을 기억하세요.
- 가중치 초기화가 다르기 때문에 출력이 약간 달라 보일 수 있지만, 두 구현 모두 이후 장에서 만들 GPT 클래스에 그대로 사용할 수 있습니다.
- 또한 `MultiHeadAttention` 클래스에 `self.out_proj` 선형 투영 층을 추가했습니다. 이는 차원을 바꾸지 않는 단순한 선형 변환으로, LLM 구현에서 흔히 사용하는 관례일 뿐 필수 요소는 아닙니다(최근 연구에서는 이를 제거해도 성능이 유지된다고 보고하고 있으며, 자세한 내용은 장 끝의 추가 읽을거리에서 확인할 수 있습니다).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- 위 내용을 간결하고 효율적으로 구현한 예시를 찾고 있다면 PyTorch에서 제공하는 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 클래스를 참고해도 좋습니다.

- 위 구현이 다소 복잡해 보일 수 있으니, `attn_scores = queries @ keys.transpose(2, 3)` 연산이 실제로 어떻게 동작하는지 살펴보겠습니다:

In [37]:
# (배치 b, 헤드 수, 토큰 수, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 이 경우 PyTorch의 행렬 곱 구현은 4차원 텐서를 처리할 때 마지막 두 차원(토큰 수, 헤드 차원)끼리 곱한 뒤, 각 헤드에 대해 해당 연산을 반복 실행합니다.

- 예를 들어 다음과 같이 각 헤드별 행렬 곱을 개별로 계산하면 더 간결하게 표현할 수 있습니다:

In [38]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# 요약 및 핵심 포인트

- [./multihead-attention.ipynb](./multihead-attention.ipynb) 노트북에는 2장에서 구현한 데이터 로더와 이 장의 멀티헤드 어텐션 클래스를 간결하게 정리해 두었습니다. 다음 장에서 GPT 모델을 학습할 때 사용할 예정입니다.
- 연습 문제 해설은 [./exercise-solutions.ipynb](./exercise-solutions.ipynb)에서 확인할 수 있습니다.